In [1]:
import pandas as pd
import importlib
import esn_tarnet_multi_treatment
import basemodel
import baseunit
import random
import json
import numpy as np
import pandas as pd
import feature_select
importlib.reload(feature_select)
importlib.reload(basemodel)
importlib.reload(baseunit)
importlib.reload(esn_tarnet_multi_treatment)

import subprocess
import pandas as pd
import numpy as np
import sys
import os
import joblib
import pickle
import multiprocessing
import ast
import json
from functools import partial
import torch
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader, get_worker_info,TensorDataset
from efin import *
from functools import partial
from esn_tarnet_multi_treatment import *
from feature_select import *
from tarnet import *


In [ ]:

def set_seed(seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
set_seed(42)


In [ ]:
def run_shell(hdfs_ls_cmd):
    print(hdfs_ls_cmd)
    result = subprocess.run(hdfs_ls_cmd, shell=True, capture_output=True, text=True)     
    if result.returncode != 0:
        print(f"Error listing files in HDFS: {result.stderr}")
    print(result)
    print(result.returncode)

In [ ]:

# 读取JSON文件并转换为字典
with open('/opt/tiger/rh2_params/custom_template_vars.json', 'r') as file:
    data = json.load(file)

# 打印字典
print(data)
is_load_data_model_feature = int(data['is_load_data_model_feature'])
data_path = data['data_path']
model_path = data['model_path']
feature_path = data['feature_path']
discrete_feature_path = data['discrete_feature_path']
discrete_size_path = data['discrete_size_path']
output_path = data['output_path']
begin = int(data['begin'])
end = int(data['end'])

print("is_load_data_model_feature:", is_load_data_model_feature)
print("data_path:", data_path)
print("model_path:", model_path)
print("feature_path:", feature_path)
print("discrete_feature_path:", discrete_feature_path)
print("discrete_size_path:", discrete_size_path)
print("begin:", begin)
print("end:", end)
model_path_list = model_path.split(',')

device = torch.device("cpu")

In [ ]:
if is_load_data_model_feature == 1:
    run_shell(f"rm -r /mnt/bd/wyx08/{data_path}")
    run_shell(f"hdfs dfs -get /home/byte_ecom_product_ds/{data_path} /mnt/bd/wyx08")

    for _ in model_path_list:
        run_shell(f"rm -r /mnt/bd/wyx08/{_}")
        run_shell(f"hdfs dfs -get  hdfs://harunava/home/byte_ecom_product_ds/{_} /mnt/bd/wyx08")

    run_shell(f"rm -r /mnt/bd/wyx08/{feature_path}")
    run_shell(f"hdfs dfs -get /home/byte_ecom_product_ds/{feature_path} /mnt/bd/wyx08")

    run_shell(f"rm -r /mnt/bd/wyx08/{discrete_feature_path}")
    run_shell(f"hdfs dfs -get /home/byte_ecom_product_ds/{discrete_feature_path} /mnt/bd/wyx08")

    run_shell(f"rm -r /mnt/bd/wyx08/{discrete_size_path}")
    run_shell(f"hdfs dfs -get /home/byte_ecom_product_ds/{discrete_size_path} /mnt/bd/wyx08")

    


In [ ]:

with open(f'/mnt/bd/wyx08/{feature_path}', 'rb') as f:
    feature_list = pickle.load(f)
print(len(feature_list))


with open(f'/mnt/bd/wyx08/{discrete_feature_path}', 'rb') as f:
    feature_list_discrete = pickle.load(f)
print(len(feature_list_discrete))


with open(f'/mnt/bd/wyx08/{discrete_size_path}', 'rb') as f:
    discrete_size_cols = pickle.load(f)
print(len(discrete_size_cols))


model_dict = {}
print(model_path_list)
for path in model_path_list:
    model = Tarnet(
    input_dim=len(feature_list), discrete_size_cols=discrete_size_cols,embedding_dim=3,share_dim=64,
                 share_hidden_dims =[256,128,64,64],
                 base_hidden_dims=[64,32,32,16],output_activation_base=torch.nn.Sigmoid(),
                 share_hidden_func = torch.nn.ELU(),base_hidden_func = torch.nn.ELU(), 
                 task = 'classification',classi_nums=2, treatment_label_list=[0,1,2,3],model_type='Tarnet',device=device
    ) 
    # 先实例化模型
    model.load_state_dict(torch.load(f'/mnt/bd/wyx08/{path}', map_location=device)['model_state_dict'])
    model_dict[path.replace('.joblib','').replace('.pt','').replace('.pth','')] = model
    
print(model_dict)


In [ ]:
import os
# 定义文件夹路径
folder_path = f'/mnt/bd/wyx08/{data_path}'

# 获取文件夹下的所有文件名
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(len(file_names))
res = []
output_cols = []

df = pd.read_parquet(folder_path+'/'+file_names[0])

if 'user_id' in list(df.columns):
    output_cols.append('user_id')

if 'device_id' in list(df.columns):
    output_cols.append('device_id')

if 'country_code' in list(df.columns):
    output_cols.append('country_code')

if 'country' in list(df.columns):
    output_cols.append('country')

output_cols = list(set(output_cols))
print(output_cols)

# 打印所有文件名
rank_ = 1
print(rank_)
import time
for file_name in file_names:
    if file_name.endswith('.parquet') and begin <= int(file_name.split('-')[1]) and int(file_name.split('-')[1]) <= end:
        start_time = time.time()
        print(rank_,file_name)
        df_tmp = pd.read_parquet(folder_path+'/'+file_name)
        print(df_tmp.shape)

        for column in feature_list:
            if df_tmp[column].dtype != 'float':
                df_tmp[column] = df_tmp[column].astype('float')

        for i in range(len(feature_list_discrete)):
            df_tmp[column] = df_tmp[column].apply(lambda x: x if x >=0 and x <= discrete_size_cols[i]-2 else discrete_size_cols[i]-1)
        
        uplift_predictions_cols = {}

        for k,model in model_dict.items():
            df = df_tmp

            X_discrete = torch.tensor(df[feature_list_discrete].values, dtype=torch.float32).to(device)
            X_continuous = torch.tensor(df[[_ for _ in feature_list if _ not in feature_list_discrete]].values, dtype=torch.float32).to(device)
            model.eval()
            with torch.no_grad(): 
                uplift_predictions,y_preds,*eps = model(None, None, X_discrete=X_discrete, X_continuous=X_continuous)
            uplift_prediction = torch.cat(uplift_predictions,dim=1).detach().cpu().numpy()

            for i in range(len(uplift_predictions)):
                if rank_ == 1:
                    output_cols.append(str(i+1)+'_score')
                df[str(i+1)+'_score'] = uplift_prediction[:,i]
            print(df[output_cols])

        rank_ += 1
        print(f"predict time: {time.time() - start_time:.4f}s")

        res.append(df[output_cols])
        del df,df_tmp
        import gc
        gc.collect()

final_res = pd.concat(res)
final_res


In [ ]:
run_shell(f"mkdir /mnt/bd/wyx08/{output_path}")
final_res.to_parquet(f'/mnt/bd/wyx08/{output_path}/{begin}_{end}.parquet', engine='pyarrow', index=False)


In [ ]:
if is_load_data_model_feature == 1:
    run_shell(f"hdfs dfs -rm -r /home/byte_ecom_product_ds/{output_path}")
run_shell(f"hdfs dfs -mkdir /home/byte_ecom_product_ds/{output_path}/")
run_shell(f"hdfs dfs -rm -r /home/byte_ecom_product_ds/{output_path}/{begin}_{end}.parquet")
run_shell(f"hdfs dfs -put -f /mnt/bd/wyx08/{output_path}/{begin}_{end}.parquet /home/byte_ecom_product_ds/{output_path}/")